RAIN - Real & Artificial Intelligence for Neuroscience

## Create models
- This notebook will create and train Artificial Neural Networks to identify exploration using rodent and target position along with manually labeled data.

#### Requirements:

- A set of position files
- Labeled data for those position files (to train the model)

or

- Access to the example file **colabels.csv**, where we can find:
    - Position and labels for representative exploration events
    - Manual labels from 5 viewers (so far)

---
#### Load the necessary modules

In [ ]:
from pathlib import Path
import rainstorm.modeling as rst

---
#### 1. State your models project path
`base` : The path to the downloaded repository.

`models_folder` : The path to the folder containing the files you'll use to create the models.

In [ ]:
base = Path.cwd()
models_folder = base / 'examples' / 'models'

---
#### Create the colabels file

The Colabels file is a csv file that contains both mice positions, exploration target positions, and one or more sets of labels for the behavior you want to analyze.

If you want to train the models using your own colabels file, you can create it using the create_colabels function below. 

All you need is:
- A folder containing the positions of the mice
- A folder for each labeler, containing the labels for the behavior you want to analyze
- A list of the targets (stationary points) present on your videos


```python
path = r'path/to/colabels_folder' # The path to the directory containing the positions folder and labelers folders
labelers = ['labeler_A', 'labeler_B', 'etc']
targets = ['tgt_1', 'tgt_2', 'etc']

rst.create_colabels(path, labelers, targets)
```

##### There is a Colabels file available in the models folder which contains positions and labels for mice on a novel object recognition task. If you want to analyze the demo data, train the models using that colabels file.

---
#### 2.  Create the modeling.yaml file

The modeling.yaml file is a configuration file that contains all the parameters needed to create and train the models. It will be located in the models folder.

In [ ]:
modeling = rst.create_modeling(models_folder, overwrite=True)


It contains the following parameters:

`path` : Path to the models folder

`colabels` : The colabels file is used to store and organize positions and labels for model training
- colabels_path: Path to the colabels folder
- labelers: List of labelers on the colabels file (as found in the columns)
- target: Name of the target on the colabels file

`focus_distance`: Window of frames to consider around an exploration event

`bodyparts`: List of bodyparts used to train the model

`split`: Parameters for splitting the data into training, validation, and testing sets
- validation: Percentage of the data to use for validation
- test: Percentage of the data to use for testing

`RNN`: Set up the Recurrent Neural Network
- width: Defines the shape of the wide model
  - past : Number of past frames to include
  - future : Number of future frames to include
  - broad : Broaden the window by skipping some frames as we stray further from the present
- units: Number of neurons on each layer
- batch_size: Number of training samples the model processes before updating its weights
- dropout: Randomly turn off a fraction of neurons in the network
- total_epochs: Each epoch is a complete pass through the entire training dataset
- initial_lr: Initial learning rate
- peak_lr: Peak learning rate
- patience: Number of epochs to wait before early stopping


---
#### 3. Before training a model, we need to prepare our training data
- First, we load the dataset from the colabels file and create one 'labels' column out of all the labelers.
- Next (optional, but recommended) we can erase the rows of the dataset that are too far away from exploration events.
- Finally, we split the dataset into training, testing and validation subsets.

In [ ]:
# Prepare the data
dataset = rst.prepare_data(modeling)

# Focus on the rows near exploratory behaviour
dataset = rst.focus(modeling, dataset)

# Split the data
model_dict = rst.split_tr_ts_val(modeling, dataset)

# Save the split
rst.save_split(modeling, model_dict)

---
If you later want to load a previous split, run:

```python
saved_split = models_folder / 'splits/split_{example_date}.h5' # Select the split you want to rescue
model_dict = rst.load_split(saved_split)
```

---
We can see on the testing data that the exploratory events happen when the nose gets close to the object

In [ ]:
rst.plot_example_data(model_dict['X_ts'], model_dict['y_ts'])

---
#### 4. With the training data ready, we can use TensorFlow to design your very first model
- It will look at the positions of one frame at a time, and try to decide if the mouse is exploring.
- If the decision is correct the architecture will be reinforced, else it will be corrected according to the learning rate.
- We will train it for some epochs (cycles through the whole dataset) and plot how the accuracy and loss evolve.
- Also, we will be validating the training using the validation split, which contains frames that were not used for training.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
import datetime
time = datetime.datetime.now()

# Build a simple neural network
model_simple = tf.keras.Sequential([
    
    # Input layer
    Input(shape=(model_dict['X_tr'].shape[1],)), 

    # Hidden layers
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    
    # Output layer
    Dense(1, activation='sigmoid')
])

# Compile the model
model_simple.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001), 
                   loss='binary_crossentropy', metrics=['accuracy'])

model_simple.summary()

#### Train the model
Each ``epoch`` is a complete pass through the entire training dataset, while the ``batch_size`` is the number of training samples the model processes before updating its weights

In [ ]:
history_simple = model_simple.fit(model_dict['X_tr'], model_dict['y_tr'],
                                  epochs=10, batch_size=128,
                                  validation_data=(model_dict['X_val'], model_dict['y_val']))

#### Plot the training and validation loss

In [ ]:
rst.plot_history(history_simple, "Simple")

#### Calculate accuracy and precision of the model

In [ ]:
y_pred_simple = model_simple.predict(model_dict['X_ts'])
metrics_simple = rst.evaluate(y_pred_simple, model_dict['y_ts'], show_report = True)

#### And finally, save the model

In [ ]:
model_name = f'{time.date()}_simple'
rst.save_model(modeling, model_simple, model_name)

---
#### 5. Now that we have a simple model trained, we can start building more complex models

To make our artificial networks as real as possible, we can let them see a sequence of frames to decide if the mouse is exploring
- Our build_RNN function will use Bidirectional LSTM layers that allow the model to take into account the temporal sequence of frames
- It also implements early stopping and learning rate scheduler mechanisms that will prevent the model from overfitting

We can control the RNN model by changing the following parameters on the modeling.yaml file:

`units` : The number of neurons on each layer of the LSTM model

`batch_size` : The number of training samples the model processes before updating its weights

`lr` : The learning rate of the model

`epochs` : The number of times the model will be trained on the entire training dataset

`past` & `future` : If you use a LSTM model, you can set the window size by saying how many frames into the past and how many into the future you want to see.

`broad` : Once you have your window size, we can broaden the window by skipping some frames as it strays further from the present.

In [ ]:
model_wide = rst.build_RNN(modeling, model_dict)

#### Train the model

In [ ]:
model_wide_name = f'{time.date()}_wide'
history_wide = rst.train_RNN(modeling, model_wide, model_dict, model_wide_name)

#### Plot the training and validation loss

In [ ]:
rst.plot_history(history_wide, model_wide_name)
rst.plot_lr_schedule(history_wide)

#### Calculate accuracy and precision of the model

In [ ]:
y_pred_wide = model_wide.predict(model_dict['X_ts_wide'])
metrics_wide = rst.evaluate(y_pred_wide, model_dict['y_ts'], show_report=True)

#### Save the wide model

In [ ]:
rst.save_model(modeling, model_wide, model_wide_name)

---
#### 6. Finally, compare the trained models

- Since we trained using the training dataset, and validated using the validation dataset... we test each model using the testing dataset.

In [ ]:
# Print the model results
print("Evaluate model vs testing data")
print(f"{metrics_simple} -> simple")
print(f"{metrics_wide} -> wide")

---
---
#### Our trained models are stored safely in our repository, with today's date.
We can:
- Continue on this notebook and evaluate the trained models.
- Skip the evaluation and go use the models on our data in [3b-Automatic_analysis](3b-Automatic_analysis.ipynb).

---


## Evaluate models

I see you've decided to continue on this notebook! You wont regret it.

One may think that the evaluation we did on the testing set is enough, and in many cases it is. However, for our purpose of finding a model that resembles the labeling of an expert, It's better to compare the performance of the model against all the manually labeled data we have.

In [ ]:
evaluation_dict = rst.build_evaluation_dict(modeling)

---
#### 7. Calculate a good reference labeler
Since we want to compare the models and the labelers, we need to create a reference labeler.

This reference could be the mean of all the labelers, but then the labelers would have an unfair advantage.

To avoid this, we choose to simultaneously create a chimera labeler and a leave-one-out-mean:
- The chimera is created by randomly selecting a labeler on each row of the data.
- The leave-one-out-mean is created by averaging the remaining labelers.

This way, we can compare the chimera to the leave-one-out-mean knowing that they are independent.

In [ ]:
chimera_dict = rst.create_chimera_and_loo_mean(evaluation_dict['manual_labels'], seed=42)

---
#### 8. Load the models & use them to label exploration on all the available data

In [ ]:
# List the models you want to evaluate
model_paths = {
    'example_simple': models_folder / 'trained_models/example_simple.keras',
    'example_wide': models_folder / 'trained_models/example_wide.keras',
    f'{time.date()}_simple': models_folder / f'trained_models/{time.date()}_simple.keras',
    f'{time.date()}_wide': models_folder / f'trained_models/{time.date()}_wide.keras',
    # Add more models as needed...
    }

In [ ]:
models_dict = rst.build_and_run_models(modeling, model_paths, evaluation_dict['position'])

In [ ]:
evaluation_dict.update(chimera_dict)
evaluation_dict.update(models_dict)
evaluation_dict = {k: v for k, v in evaluation_dict.items() if k not in {'position', 'manual_labels'}}
print(evaluation_dict.keys())  # Check the keys to confirm the additions

---
#### 9. With all the labels organized, we can evaulate the performance of each

In [ ]:
for name, pred in evaluation_dict.items():
    metrics = rst.evaluate(pred, evaluation_dict['loo_mean'])
    print(f"{metrics} -> {name}")

We can visualize the similarity between labelers using a cosine similarity plot

In [ ]:
rst.plot_cosine_sim(evaluation_dict)

And finally, run a PCA (Principal Components Analysis) to see how much each labeler resembles eachother and the mean

In [ ]:
rst.plot_PCA(evaluation_dict, make_discrete=False)

Also, we can see both the models and the labelers performance on an example video

In [ ]:
example_folder_path = base / r'examples\colabeled_video'

labelers_example = {
    "labeler_A": "Example_labeler_A.csv",
    "labeler_B": "Example_labeler_B.csv",
    "labeler_C": "Example_labeler_C.csv",
    "labeler_D": "Example_labeler_D.csv",
    "labeler_E": "Example_labeler_E.csv"
}

rst.plot_performance_on_video(example_folder_path, model_paths, labelers_example, fps = 25, 
                              bodyparts = ['nose', 'left_ear', 'right_ear', 'head', 'neck', 'body'], 
                              targets = ['obj_1', 'obj_2'], plot_tgt = "obj_2")

---
---
#### Once we get to this point, we should have selected our favorite model.
We can move on to the next notebook, [3b-Automatic_analysis](3b-Automatic_analysis.ipynb), and use the chosen model to label our position files.

---
RAINSTORM - Created on Dec 12, 2023 - @author: Santiago D'hers